In [1]:
import numpy as np
import tensorflow as tf
import emoji
import pandas as pd
import json
import pickle


from keras_preprocessing.text import Tokenizer
from keras.models import load_model, model_from_json
from keras_preprocessing.sequence import pad_sequences

np.random.seed(0)


In [2]:
model_folder = "models/training set v.3.0"

model_name = "epochs_100_layers_30_40_20_10_5"

model_json_path = f"{model_folder}/json/{model_name}.json"

model_weight_path = f"{model_folder}/weights/{model_name}.h5"


Load the model

In [3]:
# model = load_model(f"{model_folder}{model_name}.{model_extension}")
# model = keras.models.model_from_json(f"{model_folder}{model_name}.{model_extension}")


json_file = open(f"{model_json_path}", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(model_weight_path)
print("Loaded model from disk")

Loaded model from disk


Function to get emoji from class

In [4]:
emoji_dictionary = {
    0: ":red_heart:",  # Love # X
    1: ":face_with_tears_of_joy:",  # Laughter X
    2: ":grinning_face_with_big_eyes:", # Happiness # X
    3: ":loudly_crying_face:",  # Sadness # X
    4: ":smiling_face_with_heart-eyes:",  # Adoration X
    5: ":fire:",  # Excitement X
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude X
    8: ":angry_face:",  # Anger X
    9: ":thinking_face:",  # Contemplation X
}

In [5]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Add own data

In [6]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [7]:
test = [
    "I am seeing what is happening",
    "Does loading and unloading even work?",
    "Naniii",
    "Wtf is happening",
    "Im dying",
    "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA",
    "I like your hoodie",
    "Thank you for the help",
    "Your jokes are so funny",
    "I made it",
    "You are so hot",
    "I love you so much",
    "Good job!",
    "I am trying", "I want to cry", "This is just sad",
    "I love sleeeping :)",
    "Wait what is the syllabus even rip",
    "slayyy",
    "FIRE",
    "If you're happy and you know it clap your hands"
]


test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=loaded_model.layers[0].input_shape[1], padding='post', truncating='post')

See Results

In [8]:
y_pred = loaded_model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)
y_pred

1/1 [==============================] - 3s 3s/step


array([3, 3, 9, 9, 1, 9, 3, 7, 1, 3, 0, 0, 6, 3, 3, 3, 0, 9, 9, 5, 1],
      dtype=int64)

In [9]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am seeing what is happening 😭
Does loading and unloading even work? 😭
Naniii 🤔
Wtf is happening 🤔
Im dying 😂
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 🤔
I like your hoodie 😭
Thank you for the help 🙏
Your jokes are so funny 😂
I made it 😭
You are so hot ❤️
I love you so much ❤️
Good job! 👍
I am trying 😭
I want to cry 😭
This is just sad 😭
I love sleeeping :) ❤️
Wait what is the syllabus even rip 🤔
slayyy 🤔
FIRE 🔥
If you're happy and you know it clap your hands 😂


In [10]:
# Calculate accuracy
test_labels = [9,3,8,8,3,3,4,7,1,2,5,0,6,3,3,3]

correct_predictions = np.sum(y_pred == test_labels)
total_predictions = len(test_labels)
accuracy = correct_predictions / total_predictions


# Calculate loss
# Assuming you have a loss function defined or you're using a standard loss function like categorical crossentropy
loss = 0.0
for i in range(len(test_labels)):
    true_label = test_labels[i]
    predicted_value = y_pred[i]  
    loss += (true_label - predicted_value) ** 2
loss /= len(test_labels)
print(f'Test loss (MSE): {loss}')
print(f'Test accuracy: {accuracy}')

ValueError: operands could not be broadcast together with shapes (21,) (16,) 